In [2]:
import os
import pandas as pd  # type: ignore
import torch
import transformers


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("mps")

LLAMA BASE

In [4]:
model_id="meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 4/4 [00:37<00:00,  9.43s/it]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [28]:
inputs = tokenizer("What is an apple?", return_tensors='pt').to(device)
generate_ids = model.generate(inputs.input_ids, max_new_tokens=256, eos_token_id=terminators)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [29]:
print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

What is an apple? An apple is a type of fruit that grows on apple trees. It is a juicy, sweet, and crunchy fruit that is often eaten fresh or used in a variety of dishes, such as pies, salads, and smoothies. Apples come in many different varieties, each with its own unique taste, texture, and color. Some popular types of apples include Granny Smith, Red Delicious, and Gala. Apples are a good source of fiber, vitamins, and antioxidants, making them a healthy and nutritious snack.
What are the benefits of apples?
Apples are a nutrient-rich fruit that offers numerous health benefits. Some of the benefits of apples include:
1. High in Fiber: Apples are a good source of dietary fiber, which can help promote digestive health, prevent constipation, and support healthy blood sugar levels.
2. Antioxidant-Rich: Apples contain a powerful antioxidant called quercetin, which can help protect the body against free radicals and oxidative stress.
3. May Help Lower Cholesterol: The fiber, antioxidants,

LLAMA INSTRUCT

In [20]:
model_id="meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:38<00:00,  9.66s/it]


In [45]:
messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": "What is an apple?"},
    {"role": "assistant", "content": "An apple is a fruit."},
    {"role": "user", "content": "What color is it?"},
]

In [46]:
input_ids = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt"
).to(model.device)

Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


In [47]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


In [48]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [49]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=False))

<|start_header_id|>assistant<|end_header_id|>

Apples can come in a variety of colors, including:

* Red
* Green
* Yellow
* Gold
* Orange
* Brown
* Purple

Some apples may also have a combination of these colors, such as a red and green striped apple. The color of an apple can vary depending on the specific variety and ripeness of the fruit.<|eot_id|>
